In [1]:
using AbstractGPs
using TemporalGPs

Doing approximate inference in space-time GPs using pseudo-points is, as always, similar
to performing exact inference. The primary difference is that `rand` isn't available,
and the `approx_posterior_marginals` function must be used to query the approximate
posterior. This is hopefully a temporary solution, so should change at some point in the
future.

Load up the separable kernel from TemporalGPs. You need to use this to tell TemporalGPs
that you're using a separable kernel (it's not enough just to use a kernel which
happens to be separable).
RegularInTime is a data structure for inputs which allows for different spatial locations
at each point in time, and can be used with the approximate inference scheme presented
here.

In [2]:
using TemporalGPs: Separable, RegularInTime, approx_posterior_marginals

Specify a separable kernel.
The first argument is always the kernel over space, the second the kernel over time.
You can also use sums of separble kernels.

In [3]:
k = Separable(SEKernel(), Matern52Kernel());

Build a GP, and convert it to an SDE as per usual.
Use `ArrayStorage`, not `SArrayStorage`, for these kinds of GPs.

In [4]:
f = to_sde(GP(k), ArrayStorage(Float64));

Construct inputs. Spatial locations change at each point in time.
Also works with RectilinearGrids of inputs.
Times must be increasing, points in space can be anywhere.

In [5]:
N = 50;
T = 1_000;
points_in_space = [randn(N) for _ in 1:T];
points_in_time = RegularSpacing(0.0, 0.1, T);
x = RegularInTime(points_in_time, points_in_space);

Since it's not straightforward to generate samples from this GP at `x`, use a known
function, under a bit of iid noise.

In [6]:
xs = collect(x);
y = sin.(first.(xs)) .+ cos.(last.(xs)) + sqrt.(0.1) .* randn(length(xs));

Spatial pseudo-point locations.

In [7]:
z_r = range(-3.0, 3.0; length=5);

Locations in space at which to make predictions. Assumed to be the same at each point in
time, but this assumption could easily be relaxed.

In [8]:
N_pr = 150;
x_r_pr = range(-5.0, 5.0; length=N_pr);

Compute the approximate posterior marginals.

In [9]:
f_post_marginals = approx_posterior_marginals(dtc, f(x, 0.1), y, z_r, x_r_pr);
m_post_marginals = mean.(f_post_marginals);
σ_post_marginals = std.(f_post_marginals);

Visualise the posterior marginals. We don't do this during in CI because it causes
problems.

In [10]:
if get(ENV, "TESTING", "FALSE") == "FALSE"
    using Plots
    savefig(
        plot(
            heatmap(reshape(m_post_marginals, N_pr, T)),
            heatmap(reshape(σ_post_marginals, N_pr, T));
            layout=(1, 2),
        ),
        "posterior.png",
    );
end

"/home/runner/work/TemporalGPs.jl/TemporalGPs.jl/docs/src/examples/approx_space_time_inference/posterior.png"

<hr />
<h6>Package and system information</h6>
<details>
<summary>Package information (click to expand)</summary>
<pre>
Status &#96;~/work/TemporalGPs.jl/TemporalGPs.jl/examples/approx_space_time_inference/Project.toml&#96;
  &#91;99985d1d&#93; AbstractGPs v0.5.16
  &#91;98b081ad&#93; Literate v2.14.0
  &#91;91a5bcdd&#93; Plots v1.38.9
  &#91;e155a3c4&#93; TemporalGPs v0.6.3 &#96;/home/runner/work/TemporalGPs.jl/TemporalGPs.jl#564e73f&#96;
</pre>
To reproduce this notebook's package environment, you can
<a href="./Manifest.toml">
download the full Manifest.toml</a>.
</details>
<details>
<summary>System information (click to expand)</summary>
<pre>
Julia Version 1.8.5
Commit 17cfb8e65ea &#40;2023-01-08 06:45 UTC&#41;
Platform Info:
  OS: Linux &#40;x86_64-linux-gnu&#41;
  CPU: 2 × Intel&#40;R&#41; Xeon&#40;R&#41; CPU E5-2673 v4 @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 &#40;ORCJIT, broadwell&#41;
  Threads: 1 on 2 virtual cores
Environment:
  JULIA_DEBUG &#61; Documenter
  JULIA_LOAD_PATH &#61; :/home/runner/.julia/packages/JuliaGPsDocs/e8FS0/src
</pre>
</details>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*